# **ResNet-50 강의 노트**

본 노트북에서는 ResNet-50 모델에 대해 간략히 살펴보고, PyTorch를 이용해 직접 구현해보겠습니다.

## 목차
1. ResNet(Residual Network) 개요
2. ResNet-50 구조
3. Residual Block (Bottleneck) 개념도
4. PyTorch 기반 코드 구현 예시
5. 학습 방법 개요
6. 요약 및 강의 포인트

---


## 1. ResNet(Residual Network) 개요

**등장 배경**
- 딥러닝 모델이 깊어질수록 일반적으로 성능이 향상되는 경향이 있지만, 지나치게 깊은 네트워크는 기울기 소실(vanishing gradient) 등 학습 난이도가 크게 증가합니다.
- ResNet은 이러한 문제를 해결하기 위해 **잔차 학습(Residual Learning)** 이라는 개념을 도입하였습니다.

**스킵 연결(Skip Connection)과 Residual Block**
- Residual Block에서는 입력 \( x \)와 출력 \( F(x) \)을 직접 연결(스킵 연결)해 \( y = F(x) + x \) 형태로 만듭니다.
- 이 구조를 통해 깊은 네트워크에서 발생하는 기울기 소실을 완화하고, 학습 안정성을 높입니다.


## 2. ResNet-50 구조

ResNet-50은 이름에서 알 수 있듯, **50층**으로 구성된 깊은 CNN 모델입니다.

### 2.1 구성 요약
1. **초기 단계(Stem)**
   - 7×7 Conv(stride=2, 채널=64)
   - Max Pooling(3×3, stride=2)
2. **Conv2_x 블록**
   - 3개의 Bottleneck Block
3. **Conv3_x 블록**
   - 4개의 Bottleneck Block
4. **Conv4_x 블록**
   - 6개의 Bottleneck Block
5. **Conv5_x 블록**
   - 3개의 Bottleneck Block
6. **분류 헤드(Classifier)**
   - Global Average Pooling + FC(1000 클래스)

### 2.2 전체 개념도

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/0*tH9evuOFqk8F41FG.png" alt="ResNet-50 구조" width="600" />

위 그림은 ResNet-50(또는 34, 101 등의 변형)에 대한 개념적 구조를 보여줍니다. 실제로는 Bottleneck 모듈의 반복 횟수에 따라 깊이가 달라집니다.



## 3. Residual Block (Bottleneck) 개념도

아래는 Bottleneck 블록의 구조를 단순화한 개념 그림입니다.

<img src="https://i.sstatic.net/kbiIG.png" alt="Bottleneck block" width="500" />


이러한 잔차 학습 구조(\( y = F(x) + x \))를 통해 네트워크가 보다 쉽게 학습할 수 있습니다.

## 4. PyTorch 기반 코드 구현 예시

### 4.1 TorchVision 활용 (간단 예시)
가장 쉽게 ResNet-50을 사용하려면 `torchvision.models`에서 미리 정의된 모델을 가져오면 됩니다.


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# 미리 학습되지 않은(랜덤 초기화) ResNet-50
resnet50 = models.resnet50(pretrained=False)

# 미리 학습된(pretrained) ResNet-50
resnet50_pretrained = models.resnet50(pretrained=True)

# 마지막 FC 레이어 수정 (예: 클래스 수 10개로 변경)
# Imagenet으로 학습된 모델이라 마지막 레이어를 수정, 원하는 목적으로 분류하기 위해
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, 10)
print(resnet50)

### 4.2 Bottleneck 블록 직접 구현하기
학습용으로는 Bottleneck 블록을 직접 구현하여 구조를 이해해볼 수도 있습니다.


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Bottleneck(nn.Module):
    expansion = 4  # Bottleneck 확장 배수
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()

        # 1x1 Conv
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # 3x3 Conv
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 1x1 Conv (채널 확장)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        # downsample이 있으면 skip connection의 차원을 맞춤
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

### 4.3 ResNet-50 전체 구조 구현
아래는 ResNet-50 모델을 (단순화하여) 직접 구현한 예시입니다. 실제로는 더욱 세부적인 요소가 있지만, 주요 골자는 동일합니다.


In [ ]:
class MyResNet50(nn.Module):
    def __init__(self, num_classes=1000):
        super(MyResNet50, self).__init__()

        # 초기 Stem
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # inplanes 설정
        self.inplanes = 64

        # 레이어 구성
        self.layer1 = self._make_layer(Bottleneck, 64,  3, stride=1)
        self.layer2 = self._make_layer(Bottleneck, 128, 4, stride=2)
        self.layer3 = self._make_layer(Bottleneck, 256, 6, stride=2)
        self.layer4 = self._make_layer(Bottleneck, 512, 3, stride=2)

        # 분류기
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * Bottleneck.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None

        # stride!=1이거나, 채널 수가 맞지 않으면 다운샘플 구성
        if stride != 1 or self.inplanes != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = []
        layers.append(block(self.inplanes, out_channels, stride, downsample))
        self.inplanes = out_channels * block.expansion

        # 나머지 블록은 stride=1
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        # Stem
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # 4개의 레이어
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # 분류
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 모델 예시 생성
model = MyResNet50(num_classes=1000)
print(model)

## 5. 학습 방법 개요
1. **데이터셋 준비**: ImageNet, CIFAR-10 등
2. **DataLoader 설정**: 배치 크기, 셔플, 증강(augmentation) 등
3. **optimizer 및 loss 함수 설정**: 예) `torch.optim.SGD`, `nn.CrossEntropyLoss`
4. **Training Loop**:
   - 입력 → 모델 → 손실 계산 → `loss.backward()` → `optimizer.step()`
5. **Validation / Test**: 일정 에폭마다 검증 세트, 테스트 세트 정확도 확인


## 6. 요약 및 강의 포인트
- **Residual Learning**: 깊은 신경망의 기울기 소실을 완화하고 학습 효율을 개선
- **Bottleneck 구조**: 1×1 Conv → 3×3 Conv → 1×1 Conv 순으로 채널 압축 & 확장
- **ResNet-50**: 50층 깊이의 모델이며, Residual Block으로 구성
- **PyTorch 구현**: `torchvision.models.resnet50` 간편 사용, 또는 Bottleneck과 ResNet 클래스를 직접 구현
- **전이 학습**: 사전 학습된 가중치를 활용하면 적은 데이터로도 좋은 성능을 낼 수 있음

---
**추가 자료**
- [논문: Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
- [PyTorch 공식 튜토리얼](https://pytorch.org/tutorials/)
- [TorchVision 모델 문서](https://pytorch.org/vision/stable/models.html)

이상으로 **ResNet-50**에 대한 핵심 개념과 코드를 살펴보았습니다. 필요한 경우, CIFAR-10 등 소규모 데이터셋으로 실습을 진행하면서 실제 학습 과정을 경험해 보실 수 있습니다.